## 初始化信息

In [1]:
import logging
import os
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO

# 加载模型
model = YOLO('yolov8n.pt')  # 使用适合你的YOLO模型
# 禁用所有日志输出
logging.disable(logging.CRITICAL)
# 获取类别名称
class_names = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog",
    "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
    "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork",
    "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog",
    "pizza", "donut", "cake", "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv",
    "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]
class_metrics = {class_name: {'TP': 0, 'FP': 0, 'FN': 0} for class_name in class_names}


## IOU计算函数

In [2]:
def calculate_iou(pred_bbox, true_bbox, image_width, image_height):
    # 将预测框坐标从像素转换为相对坐标
    pred_x_center = pred_bbox[0]
    pred_y_center = pred_bbox[1]
    pred_width = pred_bbox[2]
    pred_height = pred_bbox[3]
    
    true_x_center = true_bbox[0] * image_width
    true_y_center = true_bbox[1] * image_height
    true_width = true_bbox[2] * image_width
    true_height = true_bbox[3] * image_height

    # 计算预测框和真实框的四个角的坐标（归一化）
    pred_x1 = pred_x_center - pred_width / 2
    pred_y1 = pred_y_center - pred_height / 2
    pred_x2 = pred_x_center + pred_width / 2
    pred_y2 = pred_y_center + pred_height / 2

    true_x1 = true_x_center - true_width / 2
    true_y1 = true_y_center - true_height / 2
    true_x2 = true_x_center + true_width / 2
    true_y2 = true_y_center + true_height / 2

    # 计算交集
    inter_x1 = max(pred_x1, true_x1)
    inter_y1 = max(pred_y1, true_y1)
    inter_x2 = min(pred_x2, true_x2)
    inter_y2 = min(pred_y2, true_y2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    pred_area = (pred_x2 - pred_x1) * (pred_y2 - pred_y1)
    true_area = (true_x2 - true_x1) * (true_y2 - true_y1)

    union_area = pred_area + true_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

def compare_results(image_file):
    image = cv2.imread(str(image_file))
    image_height, image_width, _ = image.shape  # 获取图像的宽度和高度

    # 进行YOLO推理
    results = model(image)
    
    # 获取YOLO推理的结果
    preds = results[0].boxes  # 获得框的位置和类别信息
    pred_labels = preds.cls.cpu().numpy().astype(int)
    pred_bboxes = preds.xywh.cpu().numpy()  # 获取相对坐标: [x_center, y_center, width, height]

    # 获取标注文件路径
    label_file = annotations_path / (image_file.stem + '.txt')  # 使用 Path 对象进行路径拼接
    if not label_file.exists():
        return None  # 如果没有对应的标签文件，跳过

    # 读取标注
    true_labels = []
    true_bboxes = []
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            true_labels.append(int(parts[0]))  # 类别标签
            true_bboxes.append(list(map(float, parts[1:])))  # 边界框位置（归一化的）

    # 计算IOU并判断预测结果是否正确
    total_pred = len(pred_labels)
    total_true = len(true_labels)
    
    for i in range(min(total_pred, total_true)):  # 比较最小数量的预测框和标注框
        pred_label = pred_labels[i]
        pred_bbox = pred_bboxes[i]
        true_label = true_labels[i]
        true_bbox = true_bboxes[i]

        # 检查是否类别匹配，且计算IOU
        if pred_label == true_label:
            iou = calculate_iou(pred_bbox, true_bbox, image_width, image_height)
            if iou > 0.7:  
                # 更新TP
                class_metrics[class_names[pred_label]]['TP'] += 1
            else:
                # 更新FP
                class_metrics[class_names[pred_label]]['FP'] += 1

        else:
            # 如果预测的标签与真实标签不一致，更新FP和FN
            class_metrics[class_names[pred_label]]['FP'] += 1
            class_metrics[class_names[true_label]]['FN'] += 1

    return total_pred, total_true



## 结果打印函数

In [3]:
import pandas as pd
from IPython.display import display

def display_metrics(class_labels, precision, recall, f1_score):
    """
    将 Precision, Recall, F1-Score 数据汇总到 DataFrame 并显示
    
    参数:
    class_labels (list): 类别标签列表
    precision (list): 每个类别的 Precision 列表
    recall (list): 每个类别的 Recall 列表
    f1_score (list): 每个类别的 F1-Score 列表
    """
    # 将数据汇总到 DataFrame
    data = {
        'Class': class_labels,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1_score
    }
    df = pd.DataFrame(data)
    # print(df) #需要复制结果时启用
    # 显示表格
    display(df)

# 调用示例：
# display_metrics(class_labels, precision, recall, f1_score)


## 正常光照测试

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

# 使用相对路径构建验证集路径和标注路径
val_path = Path(r'.\data\Val\high')  # 使用原始字符串避免转义问题

image_files = list(val_path.glob('*.jpg'))
print(f"Total number of images in the validation set: {len(image_files)}")

# 设置标注文件路径
annotations_path = Path(r'.\data\Annotations (high)\Labels')  # 使用原始字符串
class_metrics = {class_name: {'TP': 0, 'FP': 0, 'FN': 0} for class_name in class_names}
# 对所有图像进行评估
precision = []
recall = []
f1_score = []
class_labels = []

# 对所有图像进行评估
for image_file in image_files:
    compare_results(image_file)

# 计算每个类别的Precision, Recall, F1-Score
# 计算 Precision, Recall, F1-Score
for class_name in class_names:
    TP = class_metrics[class_name]['TP']
    FP = class_metrics[class_name]['FP']
    FN = class_metrics[class_name]['FN']

    Precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    Recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    F1_Score = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) > 0 else 0
    if Precision + Recall + F1_Score > 0:
        precision.append(Precision)
        recall.append(Recall)
        f1_score.append(F1_Score)
        class_labels.append(class_name)
# 将 Precision, Recall, F1-Score 数据汇总到 DataFrame
data = {
    'Class': class_labels,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1_score
}

df = pd.DataFrame(data)

display_metrics(class_labels, precision, recall, f1_score)

Total number of images in the validation set: 3000


,Class,Precision,Recall,F1-Score
0,person,0.987860,0.995413,0.991622
1,bicycle,1.000000,1.000000,1.000000
2,car,0.995233,0.997515,0.996373
3,motorcycle,0.983709,0.992415,0.988043
4,airplane,1.000000,1.000000,1.000000
5,bus,0.989437,0.989437,0.989437
6,train,1.000000,1.000000,1.000000
7,truck,0.997067,0.998532,0.997799
8,boat,1.000000,1.000000,1.000000
9,traffic light,1.000000,1.000000,1.000000


## 低光照测试

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

# 使用相对路径构建验证集路径和标注路径
val_path = Path(r'.\data\Val\low')  # 低光照验证集路径
image_files = list(val_path.glob('*.jpg'))
print(f"Total number of images in the validation set: {len(image_files)}")

# 设置标注文件路径
annotations_path_low = Path(r'.\data\Annotations (low)\Labels')

class_metrics = {class_name: {'TP': 0, 'FP': 0, 'FN': 0} for class_name in class_names}
# 对所有图像进行评估
precision = []
recall = []
f1_score = []
class_labels = []

# 对所有图像进行评估
for image_file in image_files:
    compare_results(image_file)

# 计算每个类别的Precision, Recall, F1-Score
# 计算 Precision, Recall, F1-Score
for class_name in class_names:
    TP = class_metrics[class_name]['TP']
    FP = class_metrics[class_name]['FP']
    FN = class_metrics[class_name]['FN']

    Precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    Recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    F1_Score = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) > 0 else 0
    if Precision + Recall + F1_Score > 0:
        precision.append(Precision)
        recall.append(Recall)
        f1_score.append(F1_Score)
        class_labels.append(class_name)
# 将 Precision, Recall, F1-Score 数据汇总到 DataFrame
data = {
    'Class': class_labels,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1_score
}

df = pd.DataFrame(data)

display_metrics(class_labels, precision, recall, f1_score)

Total number of images in the validation set: 3000


,Class,Precision,Recall,F1-Score
0,person,0.290744,0.412043,0.340925
1,bicycle,0.175926,0.231707,0.200000
2,car,0.485606,0.747873,0.588858
3,motorcycle,0.206236,0.272596,0.234818
4,airplane,0.428571,0.500000,0.461538
5,bus,0.396907,0.427184,0.411490
6,train,0.200000,0.384615,0.263158
7,truck,0.394471,0.442457,0.417088
8,traffic light,0.242683,0.262187,0.252058
9,fire hydrant,0.416667,0.238095,0.303030
